In [1]:
import os
import getpass
from pathlib import Path
from dotenv import load_dotenv

dotenv_path = Path('.env')
if not dotenv_path.exists():
    dotenv_path = Path('..') / '.env'
load_dotenv(dotenv_path=dotenv_path)

required_vars = ["GEMINI_API_KEY"]
for var in required_vars:
    if not os.getenv(var):
        os.environ[var] = getpass.getpass(f"Enter {var}: ")

# Avoid provider key conflicts in mixed environments.
if os.getenv("GEMINI_API_KEY"):
    os.environ.pop("GOOGLE_API_KEY", None)


# LangChain LLM Chain Tutorial 

## Environment Setup
Install dependencies using `requirements.txt` as described in the repository README.

## Build a basic agent


In [2]:
from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Herramienta de ejemplo."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model="google_genai:gemini-2.5-flash",   
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "what is the weather in sf"}]
})
print(result)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='949bdfe7-d69b-4dd2-8fbd-5a8a6b0c0c2b'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "San Francisco"}'}, '__gemini_function_call_thought_signatures__': {'38edd0ca-f20b-4bf2-8013-ee90e60b9cd4': 'CocCAb4+9vulKkfAD9Hc53Sllyxhpd/4tn22rWz3XP6bMpWbRGeU6ffgHjxcuko0mcNuImu9wMTDdxdSgm5M1XNQm2tN/pd2WTEiBUQ3zZ5ItrcxCai3dVdgbei3p6ugamSU0CJYkJTYJhn+aQOTvRkE2nTtSW0VuDjK2mimtmWqLNLwJdt0b0F8vnF87qUH/VeFNTuXh0WvCigzrqCva78NYf374mKt+axEHXxfktqCsFpoCEtZmDVK7z+cFle+rTof/KxsFEV8/wmq8B4gNxRb1mb5sZK02rTElcfwOvd6Zte4Bt13v6qi/di4ic5PNkiumPpBjyDz5la3mfn2khbD2n/ZGzWphd0='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c78a7-dc73-72f3-a7ee-4a6e73fe4cae-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': '38edd0c

# Build a real-world agent

### Build a Practical Weather Forecasting Agent

This example demonstrates key production concepts:

1. **Detailed system prompts** for better agent behavior
2. **External data integration** via custom tools
3. **Model configuration** for consistent responses
4. **Structured output** for predictable results
5. **Conversational memory** for chat-like interactions
6. **End-to-end testing** by running the fully functional agent

Let's walk through each step:

## 1. Define the system prompt

The system prompt defines your agent’s role and behavior. Keep it specific and actionable:

In [3]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

## 2. Create tools


Tools let a model interact with external systems by calling functions you define. Tools can depend on runtime context and also interact with agent memory.
Notice below how the get_user_location tool uses runtime context

In [4]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

## 3. Configure the model
Set up the language model with the right parameters for your use case:

In [5]:
import os
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "google_genai:gemini-2.5-flash",
    api_key=os.environ["GEMINI_API_KEY"],
    temperature=0.5,
    timeout=10,
    max_tokens=1000,
    max_retries=2,
)


## 4.Define response format
Optionally, define a structured response format if you need the agent responses to match a specific schema.


In [6]:
from dataclasses import dataclass

# We use a dataclass here, but Pydantic models are also supported.
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

## 5. Add memory

Add memory to your agent to maintain state across interactions. This allows the agent to remember previous conversations and context.

In [7]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

## 6. Create and run the agent
Now assemble your agent with all the components and run it!

In [8]:
import os
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

model = init_chat_model(
    "google_genai:gemini-2.5-flash",
    api_key=os.environ["GEMINI_API_KEY"],
    temperature=0.3,
    timeout=20,
    max_tokens=800,
)

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer,
)

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1"),
)

print(response["structured_response"])


ResponseFormat(punny_response='Looks like the weather in Florida is quite a *bright* spot! Hope you have a *sun-sational* day!', weather_conditions="It's always sunny in Florida!")
